<a href="https://colab.research.google.com/github/KabichNeu/Hands-on-Machine-Learning/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')

In [8]:
f = x*x*y + y + 2

In [10]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)


42


In [12]:
sess.close()

In [13]:
#ANOTHER WAY OF DOING THIS

In [14]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()


In [15]:
print(result)

42


In [16]:
#Instead of manually running the initializer for every single variable, you can use the global_variables_initializer() function.

In [17]:
init = tf.global_variables_initializer()   #prepare an init node

with tf.Session() as sess:
  init.run()     #actually initialize all the variables
  result = f.eval()

print(result)

42


In [18]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)


42


In [19]:
sess.close()

In [20]:
#managing the graphs

In [21]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [22]:
#if you want to create multiple independent graph

In [23]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)

x2.graph is graph

True

In [24]:
x2.graph is tf.get_default_graph()

False

In [26]:
#to reset default graph
tf.reset_default_graph()

In [27]:
#Lifecycle of a Node Value
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
  print(y.eval())
  print(z.eval())

10
15


In [28]:
#If you want to evaluate y and z efficiently, without evaluating w and x twice as in the
#previous code, you must ask TensorFlow to evaluate both y and z in just one graph
#run.

In [30]:
with tf.Session() as sess:
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)

10
15


In [32]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data] #Translates slice objects to concatenation along the second axis.

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
  theta_value = theta.eval()

In [33]:
#Implementing Batch Gradient Descent

#manually computing the gradients

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32,name= 'y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name= "theta")
y_pred = tf.matmul(X, theta, name = "prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    if epoch %100 ==0:
      print("Epoch",  epoch, "MSE = ", mse.eval())
    sess.run(training_op)

  best_theta = theta.eval()


Epoch 0 MSE =  6.9127655
Epoch 100 MSE =  0.74084187
Epoch 200 MSE =  0.62077457
Epoch 300 MSE =  0.59401333
Epoch 400 MSE =  0.57557815
Epoch 500 MSE =  0.5621355
Epoch 600 MSE =  0.5522994
Epoch 700 MSE =  0.5450873
Epoch 800 MSE =  0.53978753
Epoch 900 MSE =  0.53588355


In [37]:
best_theta

array([[ 2.0685523 ],
       [ 0.84008086],
       [ 0.15155753],
       [-0.22760165],
       [ 0.24896424],
       [ 0.00729534],
       [-0.04232442],
       [-0.65091497],
       [-0.6199971 ]], dtype=float32)

In [39]:
#Using autodifference->it can automatically
#and efficiently compute the gradients for you

In [40]:
gradients = tf.gradients(mse, [theta])[0]

In [41]:
#using an optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

In [42]:
#If you want to use a different type of optimizer, you just need to change one line. For
#example, you can use a momentum optimizer

optimizer = tf.train.MomentumOptimizer(learning_rate  = learning_rate, momentum = 0.9)

In [43]:
#Feeding Data to the Training Algorithm

In [44]:
#implementing mini gradient descent 
#we will be using placeholders

In [45]:
X = tf.placeholder(tf.float32, shape = (None, n+1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1), name = "y")


batch_size = 100
n_batches = int(np.ceil(m/ batch_size))   #Return the ceiling of the input, element-wise.

def fetch_batch(epoch, batch_index, batch_size):
  np.random.seed(epoch*n_batches + batch_index)
  indices = np.random.randint(m, size = batch_size)
  X_batch = scaled_housing_data_plus_bias[indices]
  y_batch = housing.target.reshape(-1,1)[indices]
  return X_batch, y_batch

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
      sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
  best_theta = theta.eval()


In [46]:
best_theta

array([[ 2.0685525 ],
       [ 0.82963324],
       [ 0.11875443],
       [-0.26555082],
       [ 0.3057157 ],
       [-0.00450222],
       [-0.03932687],
       [-0.8998529 ],
       [-0.8705097 ]], dtype=float32)

In [49]:
#saving and restoring models

n_epochs = 1000

learning_rate = 0.01

X= tf.constant(scaled_housing_data_plus_bias, dtype= tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')

theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = "prediction")

error =  y_pred - y

mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op  = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(init)

  for epoch in range(n_epochs):
    if epoch % 100 ==0:
      print("epoch", epoch, "MSE", mse.eval())
      
      save_path = saver.save(sess, "/tmp/my_model.ckpt")
    sess.run(training_op)
  best_theta = theta.eval()
  save_path = saver.save(sess,"/tmp/my_model_final.ckpt")

epoch 0 MSE 13.888548
epoch 100 MSE 0.7614644
epoch 200 MSE 0.5687348
epoch 300 MSE 0.5571052
epoch 400 MSE 0.55069077
epoch 500 MSE 0.54566574
epoch 600 MSE 0.54165596
epoch 700 MSE 0.53844047
epoch 800 MSE 0.5358518
epoch 900 MSE 0.53375995


In [50]:
#restoring model
with tf.Session() as sess:
  saver.restore(sess, "/tmp/my_model_final.ckpt")
  best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [51]:
np.allclose(best_theta, best_theta_restored)


True

In [52]:
saver = tf.train.Saver({"weights": theta})